## Scientific Computation Lab 2

#### Task 1: Timing code

There are a few different approaches for timing code in Python.
You can use the *time* function in the time module as below:

In [2]:
import numpy as np
import time
X = list(np.random.randint(1,1000,800)) 
t1 = time.time()
y = np.sin(np.outer(X,X))
t2 = time.time()
print("dt = ",t2-t1)

dt =  0.018823862075805664


And you can also use the timeit function:

In [3]:
timeit np.sin(np.outer(X,X))

6.95 ms ± 28.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


There are a few important points to be aware of when working with timing functions. First, different timers can have different resolutions, i.e. the smallest $\Delta t$ that can be measured can depend on the timing function used. Second, for very fast operations, timing results can vary substantially from one run to the next and averaging over several runs (as timeit does) can provide more reliable results. Finally, the resolution of a timing function can depend on the operating system you are using. You should check that you can reliably measure times below 1e-3 seconds with time.time(). If you cannot, you should try one or more of the other timers available in the time module (e.g. time.perf_counter_ns()). See https://docs.python.org/3/library/time.html#functions and https://docs.python.org/3/library/timeit.html for more information.

You will now investigate how the wall-time for sorting algorithms depends on the list length. Functions for merge and insertion sorts are below.

In [4]:
def isort(X):
    """Sort X using insertion sort algorithm and return sorted array
    """

    S = X.copy()

    for i,x in enumerate(X[1:],1):
        #place x appropriately in partially sorted array, S
        for j in range(i-1,-1,-1):
            if S[j+1]<S[j]:
                S[j],S[j+1] = S[j+1],S[j]
            else:
                break
    return S

def merge(L,R):
    """Merge 2 sorted lists provided as input
    into a single sorted list
    """
    M = [] #Merged list, initially empty
    indL,indR = 0,0 #start indices
    nL,nR = len(L),len(R)

    #Add one element to M per iteration until an entire sublist
    #has been added
    for i in range(nL+nR):
        if L[indL]<R[indR]:
            M.append(L[indL])
            indL = indL + 1
            if indL>=nL:
                M.extend(R[indR:])
                break
        else:
            M.append(R[indR])
            indR = indR + 1
            if indR>=nR:
                M.extend(L[indL:])
                break
    return M

def mergesort(X):
    """Given a unsorted list, sort list using
    merge sort algorithm and return sorted list
    """

    n = len(X)

    if n==1:
        return X
    else:
        L = mergesort(X[:n//2])
        R = mergesort(X[n//2:])
        return merge(L,R)

1) How much does the wall-time for insertion sort increase when the length of the initial, unsorted list is doubled? What do you expect? Try a few different list lengths.

In [5]:
#Add code below
#Run isort with N=4000
N=4000
X = list(np.random.randint(1,2*N,N))
t1 = time.time()
S = isort(X)
t2 = time.time()
dt1 = t2-t1
print("N=%d, dt=%f"%(N,dt1))
X2 = list(np.random.randint(1,4*N,2*N))
t1 = time.time()
S = isort(X2)
t2 = time.time()
dt2 = t2-t1
print("N=%d, dt=%f"%(2*N,dt2))
print("dt2/dt1=%f"%(dt2/dt1))

N=4000, dt=0.508052
N=8000, dt=2.061706
dt2/dt1=4.058060


Since insertion sort requires $O(N^2)$ time, we expect the wall-time to increase by a factor of 4 when $N$ is doubled. 

2) Now carry out a similar examination of the mergesort code. Note that it can be difficult to find good agreement with the theoretical cost with this approach where $N$ is simply doubled, and it is better to examine the variation of cost with $N$ more carefully than what we did for insertion sort.

In [6]:
#Add code below
pvals = [4,6,8,12,16]
avals = [5,5,5,5,5]
tvals = []
import numpy as np
for p,a in zip(pvals,avals):
    N1 = 2**p
    N2 = 2**(p+a)
    print("p=",p,"a=",a,"N1=",N1, "N2=",N2)
    X = list(np.random.randint(1,2*N2,N2))
    t1 = time.time()
    S = mergesort(X[:N1])
    t2 = time.time()
    dt1 = t2-t1
    t1 = time.time()
    S = mergesort(X)
    t2 = time.time()
    dt2 = t2-t1
    print("time ratio=",dt2/dt1," 2^a(p+a)/p=",2**a*(p+a)/p)


p= 4 a= 5 N1= 16 N2= 512
time ratio= 23.238095238095237  2^a(p+a)/p= 72.0
p= 6 a= 5 N1= 64 N2= 2048
time ratio= 45.38194444444444  2^a(p+a)/p= 58.666666666666664
p= 8 a= 5 N1= 256 N2= 8192
time ratio= 48.67391304347826  2^a(p+a)/p= 52.0
p= 12 a= 5 N1= 4096 N2= 131072
time ratio= 42.10678032044707  2^a(p+a)/p= 45.333333333333336
p= 16 a= 5 N1= 65536 N2= 2097152
time ratio= 40.85279009565891  2^a(p+a)/p= 42.0


Let's say that the cost of merge sort is $C(N) \approx cNlog_2(N)$ for large $N$ where $c$ is obtained by carrying out an operation count. Now consider the ratio of costs when $N=2^p$ and $N=2^{p+a}$ where $p$ and $a$ are positive integers. This ratio will be $2^a (p+a)/p$. So if $p\gg a$, we get the same ratio as if we had assumed a linear dependence on $N$. In the code above we fix $a=5$, so a linear-time algorithm should produce a ratio of $32$. From the output for $p>4$, we see that the ratio is larger than 16 and agrees fairly well with the expected results (though the level of agreement can vary from one run to the next). For $p=4$, the measured ratio is not at all close to what is expected. This is probably due to $N$ not being sufficiently large and the details of how data is being stored in the computer's memory being particularly important. 
There are different ways to approach this problem. A slightly different approach is shown in the cell below.

In [7]:
Na = np.array([10000,20000,40000,80000,100000,200000,400000]) #list sizes
NLN = Na*np.log2(Na) #theoretical cost (asymptotic running time)
times = []
for i,N in enumerate(Na):
    X = list(np.random.randint(1,2*N,N))
    t1 = time.time()
    S = mergesort(X)
    t2 = time.time()
    dt1m = t2-t1
    times.append(dt1m)
    print("N=%d, dt=%f"%(N,dt1m))
    if N>Na[0]:
        print("measured ratio=%f, theoretical ratio=%f"%(times[i]/times[i-1],NLN[i]/NLN[i-1]))


N=10000, dt=0.082136
N=20000, dt=0.037317
measured ratio=0.454336, theoretical ratio=2.150515
N=40000, dt=0.078625
measured ratio=2.106938, theoretical ratio=2.139980
N=80000, dt=0.166657
measured ratio=2.119641, theoretical ratio=2.130824
N=100000, dt=0.211195
measured ratio=1.267243, theoretical ratio=1.274706
N=200000, dt=0.449563
measured ratio=2.128661, theoretical ratio=2.120412
N=400000, dt=0.950294
measured ratio=2.113816, theoretical ratio=2.113574


3) Iterative merge sort

(a) We have seen that both recursive and iterative implementations of binary search are possible. What about merge sort? Write an outline for what an iterative implementation of merge sort would look like. For convenience, assume that the length of the input is $2^N$ where $N$ is a positive integer. 

**Outline:**
* Convert the input list into a list of 1-element lists, Lm (the cost of this is $O(N)$. 
* Then, while len(Lm)>1, iterate through indices i=0:-1:2 of Lm and replace Lm[i] and Lm[i+1] with merge(Lm[i],Lm[i+1])

This would still be $O(Nlog_2N)$ like the recursive version since the there are $log_2N$ steps and $O(N)$ merge operations per step.

(b) Now ask chatGPT-4o or Claude 3.5 Sonnet to create a Python function for iterative merge sort. Are there important differences between your outline and the generated function?

In [1]:
# Code in this cell generated by Claude 3.5 Sonnet #
def merge_sort(arr):
    # Base case: if the array has 1 or fewer elements, it's already sorted
    if len(arr) <= 1:
        return arr
    
    # Start merging with a minimum size of 1, doubling the size each iteration
    size = 1
    while size < len(arr):
        for start in range(0, len(arr), size * 2):
            mid = start + size
            end = min(start + size * 2, len(arr))
            
            # Merge two sorted subarrays
            left = arr[start:mid]
            right = arr[mid:end]
            merged = []
            i, j = 0, 0
            
            while i < len(left) and j < len(right):
                if left[i] <= right[j]:
                    merged.append(left[i])
                    i += 1
                else:
                    merged.append(right[j])
                    j += 1
            
            merged.extend(left[i:])
            merged.extend(right[j:])
            
            # Place the merged subarray back into the original array
            arr[start:end] = merged
        
        size *= 2
    
    return arr

The general approach is similar to the outline above. The code does not explicitly break the original list into $N$ parts. Instead it uses the size  variable to keep track of what the "current" size of the sub-lists is for each iteration, and then sets mid and end to extract the needed sub-lists that are to be merged.

In [13]:
N=11
X = list(np.random.randint(1,2*N,N))
merge_sort(X)

[1, 6, 8, 9, 10, 11, 12, 13, 13, 14, 18]

####  Task 2: Setting up a word count
In this task, you will place the words from a text file into a Python dictionary so that word counts for specified words can be quickly extracted. The cell below will read in the file (if the notebook and the text file, *words.txt*, are in the same folder).

In [8]:
infile = open('words.txt','r')
A = infile.read()
#Remove punctuation
A = A.replace(',','')
A = A.replace('?','')
A = A.replace('.','')
A = A.replace('!','')
A = A.lower() #Convert to all lowercase
words = A.split() #List (mostly) containing the words in the file
print(words[:4])
infile.close()

['sing', 'michael', 'sing', 'on']


1) Fill the dictionary below so that each word in *words* is stored as a key and the corresponding value is the number of times the word appears in the list

In [9]:
D = {} #Initializes empty dictionary
#Add code below
for w in words:
    if w in D:
        D[w] = D[w]+1
    else:
        D[w]=1

2) Using your dictionary, look up how many times the word "rudie" appears in the file 

In [10]:
#Add code here
print(D['rudie'])
print('check:',words.count('rudie'))

19
check: 19


3) In general (i.e. not specifically for this file) what are the time complexities for a) creating the dictionary and b) looking up a word count?

Add answer here:

a) Creating the dictionary requires one key lookup per word and either an insertion  or an addition, both of which are assumed to require constant time, so the time complexity will be $O(N)$ for $N$ words. (key lookup is also constant time)

b) Key lookup and accessing the corresponding value are both constant time, so the total time complexity is $O(1)$